# AAGD

In [ ]:
import numpy as np
import plotly.graph_objects as go

A = np.array([[3, 1],
              [1, 2]])
b = np.array([1, 1])

def f(x):
    return 0.5 * x @ A @ x - b @ x

def grad_f(x):
    return A @ x - b

x_star = np.linalg.solve(A, b)

# --- AAGD алгоритм ---
def adaptive_accelerated_gd(f, grad_f, x0, num_iter=50, lambda0=1.0, Lambda0=1.0):
    x0 = np.array(x0)
    x_vals = [x0.copy()]
    y_vals = []

    x_prev = x0
    y_curr = x0 - lambda0 * grad_f(x0)
    x_curr = y_curr.copy()

    x_vals.append(x_curr)
    y_vals.append(y_curr)

    lambda_k = lambda0
    Lambda_k = Lambda0
    theta_k = np.inf
    Theta_k = np.inf

    for k in range(2, num_iter + 1):
        lambda_k_1 = lambda_k
        Lambda_k_1 = Lambda_k

        diff_x = x_curr - x_prev
        diff_grad = grad_f(x_curr) - grad_f(x_prev)

        norm_ratio = np.linalg.norm(diff_x) / max(1e-8, np.linalg.norm(diff_grad))
        lambda_k = min(np.sqrt(1 + theta_k / 2) * lambda_k, norm_ratio / 2)

        grad_norm_ratio = np.linalg.norm(diff_grad) / max(1e-8, np.linalg.norm(diff_x))
        Lambda_k = min(np.sqrt(1 + Theta_k / 2) * Lambda_k, grad_norm_ratio / 2)

        sqrt_lambda = np.sqrt(1 / lambda_k)
        sqrt_Lambda = np.sqrt(Lambda_k)

        beta_k = (sqrt_lambda - sqrt_Lambda) / (sqrt_lambda + sqrt_Lambda)

        y_next = x_curr - lambda_k * grad_f(x_curr)
        x_next = y_next + beta_k * (y_next - y_curr)

        theta_k = lambda_k / lambda_k_1
        Theta_k = Lambda_k / Lambda_k_1

        x_prev = x_curr
        x_curr = x_next
        y_curr = y_next

        x_vals.append(x_curr.copy())
        y_vals.append(y_curr.copy())

    return np.array(x_vals), np.array(y_vals)

x0 = [4.0, 4.0]
x_vals, _ = adaptive_accelerated_gd(f, grad_f, x0, num_iter=40)

X = np.linspace(-6, 6, 100)
Y = np.linspace(-6, 6, 100)
X, Y = np.meshgrid(X, Y)
Z = 0.5 * (A[0,0]*X**2 + 2*A[0,1]*X*Y + A[1,1]*Y**2) - b[0]*X - b[1]*Y

fig = go.Figure()

fig.add_trace(go.Surface(x=X, y=Y, z=Z, opacity=0.7, colorscale='Viridis', name='f(x)'))

z_vals = [f(xy) for xy in x_vals]
fig.add_trace(go.Scatter3d(x=x_vals[:, 0], y=x_vals[:, 1], z=z_vals,
                           mode='lines+markers',
                           marker=dict(size=4, color='red'),
                           line=dict(color='red', width=4),
                           name='Trajectory'))

fig.add_trace(go.Scatter3d(x=[x_star[0]], y=[x_star[1]], z=[f(x_star)],
                           mode='markers',
                           marker=dict(size=8, color='green', symbol='x'),
                           name='Minimum (analytical)'))

fig.update_layout(title='AAGD for f(x) = 0.5 xᵀAx - bᵀx with Minimum',
                  scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='f(x)'),
                  width=800, height=600)

fig.show()


# Improved AAGD

In [ ]:
import numpy as np
import plotly.graph_objects as go

A = np.array([[3, 1],
              [1, 2]])
b = np.array([1, 1])

def f(x):
    return 0.5 * x @ A @ x - b @ x

def grad_f(x):
    return A @ x - b

x_star = np.linalg.solve(A, b)

L = np.linalg.eigvalsh(A).max()

def adaptive_accelerated_gd_quadratic(f, grad_f, x0, num_iter=50, lambda0=1.0, Lambda0=1.0):
    x0 = np.array(x0)
    x_vals = [x0.copy()]
    y_vals = []

    x_prev = x0
    y_curr = x0 - lambda0 * grad_f(x0)
    x_curr = y_curr.copy()

    x_vals.append(x_curr)
    y_vals.append(y_curr)

    lambda_k = lambda0
    Lambda_k = Lambda0
    theta_k = np.inf
    Theta_k = np.inf

    for k in range(2, num_iter + 1):
        lambda_k_1 = lambda_k
        Lambda_k_1 = Lambda_k

        lower_bound = 1 / (2 * L)

        lambda_k = min(np.sqrt(1 + theta_k / 2) * lambda_k, lower_bound)
        Lambda_k = min(np.sqrt(1 + Theta_k / 2) * Lambda_k, L / 2)

        sqrt_lambda = np.sqrt(1 / lambda_k)
        sqrt_Lambda = np.sqrt(Lambda_k)

        beta_k = (sqrt_lambda - sqrt_Lambda) / (sqrt_lambda + sqrt_Lambda)

        y_next = x_curr - lambda_k * grad_f(x_curr)
        x_next = y_next + beta_k * (y_next - y_curr)

        theta_k = lambda_k / lambda_k_1
        Theta_k = Lambda_k / Lambda_k_1

        x_prev = x_curr
        x_curr = x_next
        y_curr = y_next

        x_vals.append(x_curr.copy())
        y_vals.append(y_curr.copy())

    return np.array(x_vals), np.array(y_vals)

x0 = [4.0, 4.0]
x_vals, _ = adaptive_accelerated_gd_quadratic(f, grad_f, x0, num_iter=40)

X = np.linspace(-6, 6, 100)
Y = np.linspace(-6, 6, 100)
X, Y = np.meshgrid(X, Y)
Z = 0.5 * (A[0,0]*X**2 + 2*A[0,1]*X*Y + A[1,1]*Y**2) - b[0]*X - b[1]*Y

fig = go.Figure()

fig.add_trace(go.Surface(x=X, y=Y, z=Z, opacity=0.7, colorscale='Viridis', name='f(x)'))

z_vals = [f(xy) for xy in x_vals]
fig.add_trace(go.Scatter3d(x=x_vals[:, 0], y=x_vals[:, 1], z=z_vals,
                           mode='lines+markers',
                           marker=dict(size=4, color='red'),
                           line=dict(color='red', width=4),
                           name='Trajectory'))

fig.add_trace(go.Scatter3d(x=[x_star[0]], y=[x_star[1]], z=[f(x_star)],
                           mode='markers',
                           marker=dict(size=8, color='green', symbol='x'),
                           name='Minimum (analytical)'))

fig.update_layout(title='AAGD with Quadratic Step Size Adaptation',
                  scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='f(x)'),
                  width=800, height=600)

fig.show()


# Improved section 1

In [ ]:
import numpy as np
import plotly.graph_objects as go

A = np.array([[3, 1],
              [1, 2]])
b = np.array([1, 1])

def f(x):
    return 0.5 * x @ A @ x - b @ x

def grad_f(x):
    return A @ x - b

L = np.linalg.eigvalsh(A).max()

def gradient_descent_adaptive(f, grad_f, x0, num_iter=50, lambda0=1.0):
    x_vals = [x0.copy()]
    x_prev = x0.copy()
    x_curr = x0 - lambda0 * grad_f(x0)
    x_vals.append(x_curr.copy())

    lambda_k = lambda0
    theta_k = np.inf

    for k in range(2, num_iter + 1):
        lambda_k_1 = lambda_k
        lower_bound = 1 / (2 * L)
        lambda_k = min(np.sqrt(1 + theta_k / 2) * lambda_k, lower_bound)
        x_next = x_curr - lambda_k * grad_f(x_curr)
        theta_k = lambda_k / lambda_k_1
        x_prev = x_curr
        x_curr = x_next
        x_vals.append(x_curr.copy())

    return np.array(x_vals)

x0 = np.array([4.0, 4.0])
x_vals = gradient_descent_adaptive(f, grad_f, x0, num_iter=40)

x_star = np.linalg.solve(A, b)

x_range = np.linspace(-6, 6, 100)
y_range = np.linspace(-6, 6, 100)
X, Y = np.meshgrid(x_range, y_range)
Z = 0.5 * (A[0,0]*X**2 + 2*A[0,1]*X*Y + A[1,1]*Y**2) - b[0]*X - b[1]*Y

trajectory = np.array(x_vals)
Z_traj = np.array([f(p) for p in trajectory])

fig = go.Figure(data=[
    go.Surface(z=Z, x=X, y=Y, colorscale='Viridis', opacity=0.8, showscale=False),
    go.Scatter3d(x=trajectory[:, 0], y=trajectory[:, 1], z=Z_traj,
                 mode='lines+markers', line=dict(color='red', width=4),
                 marker=dict(size=4), name='Trajectory'),
    go.Scatter3d(x=[x_star[0]], y=[x_star[1]], z=[f(x_star)],
                 mode='markers', marker=dict(size=7, color='green', symbol='x'),
                 name='Analytical Minimum')
])

fig.update_layout(title='Adaptive Gradient Descent on Quadratic Function',
                  scene=dict(xaxis_title='x1', yaxis_title='x2', zaxis_title='f(x)'))

fig.show()
